In [3]:
import psycopg2
import pandas as pd

In [64]:
def create_database():
    conn = psycopg2.connect("host=127.0.0.1 dbname=postgres user=postgres password=1234")
    cur = conn.cursor()
    conn.set_session(autocommit=True)

    cur.execute("CREATE DATABASE accounts")

    conn.close()

    conn = psycopg2.connect("host=127.0.0.1 dbname=accounts user=postgres password=1234")
    cur = conn.cursor()
    conn.set_session(autocommit=True)

    return conn, cur

In [65]:
AccountsCountry = pd.read_csv("./Wealth-AccountsCountry.csv")
AccountsData = pd.read_csv("./Wealth-AccountData.csv")
AccountsSeries = pd.read_csv("./Wealth-AccountSeries.csv")

In [66]:
AccountsCountry = AccountsCountry[['Code', 'Short Name', 'Long Name', 'Income Group']]

In [67]:
AccountsData = AccountsData[['Country Code', 'Country Name', 'Series Code', '2017 [YR2017]', '2018 [YR2018]']]

In [68]:
AccountsSeries = AccountsSeries[['Code', 'Topic', 'Indicator Name']]

In [69]:
AccountsCountry.head()

,Code,Short Name,Long Name,Income Group
0,ALB,Albania,Republic of Albania,Upper middle income
1,ARG,Argentina,Argentine Republic,Upper middle income
2,ARM,Armenia,Republic of Armenia,Upper middle income
3,AUS,Australia,Commonwealth of Australia,High income
4,AUT,Austria,Republic of Austria,High income


In [70]:
AccountsData.head()

,Country Code,Country Name,Series Code,2017 [YR2017],2018 [YR2018]
0,ALB,Albania,NW.HCA.TO,7.80E+10,8.12E+10
1,ALB,Albania,NW.HCA.PC,27160.86,28333.68
2,ALB,Albania,NW.HCA.FEMP.PC,3178.417,3312.927
3,ALB,Albania,NW.HCA.MEMP.PC,18445.94,19245.19
4,ALB,Albania,NW.HCA.FEMA.PC,3992.188,4161.137


In [71]:
AccountsSeries.head()

,Code,Topic,Indicator Name
0,NW.HCA.TO,Human capital,Human capital (constant 2018 US$)
1,NW.HCA.PC,Human capital,Human capital per capita (constant 2018 US$)
2,NW.HCA.FEMP.PC,Human capital,"Human capital per capita, employed female (con..."
3,NW.HCA.MEMP.PC,Human capital,"Human capital per capita, employed male (const..."
4,NW.HCA.FEMA.PC,Human capital,"Human capital per capita, female (constant 201..."


In [72]:
conn, cur = create_database()

In [74]:
cur.execute("""CREATE TABLE IF NOT EXISTS account_country(
country_code VARCHAR,
short_name VARCHAR,
long_name VARCHAR,
income_group VARCHAR
)""")

In [77]:
cur.execute("""CREATE TABLE IF NOT EXISTS account_data(
country_code VARCHAR,
country_name VARCHAR,
series_code VARCHAR,
y_2017 numeric,
y_2018 numeric
)""")

In [78]:
cur.execute("""CREATE TABLE IF NOT EXISTS account_series(
series_code VARCHAR,
topic VARCHAR,
indicator_name VARCHAR
)""")

In [80]:
account_country_insert = """INSERT INTO account_country(
country_code,
short_name,
long_name,
income_group)
VALUES (%s, %s, %s, %s)
"""

In [83]:
for i, row in AccountsCountry.iterrows():
    cur.execute(account_country_insert, list(row))

In [85]:
account_data_insert = """INSERT INTO account_data(
country_code,
country_name,
series_code,
y_2017,
y_2018)
VALUES (%s, %s, %s, %s, %s)
"""

In [86]:
for i, row in AccountsData.iterrows():
    cur.execute(account_data_insert, list(row))

InvalidTextRepresentation: invalid input syntax for type numeric: ".."
LINE 7: VALUES ('BLZ', 'Belize', 'NW.NCA.SAGA.PC', '0', '..')
                                                        ^


In [91]:
account_series_insert = """INSERT INTO account_series(
series_code,
topic,
indicator_name)
VALUES (%s, %s, %s)
"""

In [92]:
for i, row in AccountsSeries.iterrows():
    cur.execute(account_series_insert, list(row))